# Mapa temático de deforestación

# 1. Autenticación de Google Earth Engine

Se utiliza la API de Python de GEE para buscar y descargar imágenes del satélite Sentinel-2. 
`ee`: API de Google Earth Engine para Python, que permite acceder a colecciones de datos satelitales y ejecutar operaciones en la nube.  

In [2]:
# Importar biblioteca de Google Earth Engine
import ee

# Autenticación: Esto abre una ventana en el navegador solicitando acceso a cuenta Google
ee.Authenticate()  
# Inicializar la sesion de Earth Engine
ee.Initialize()

# 2. Importar paqueterías

In [3]:
import geemap
import ee_extra
import math
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap,BoundaryNorm
from matplotlib.patches import Patch
import rasterio
import folium
import json
import os
from shapely.geometry import mapping, Polygon, MultiPolygon

C:\Users\zaira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geemap\conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# 3. Definir polígono de interés

En este bloque se establece la ruta local del **polígono de interés (area of interes, AOI)** desde un archivo en formato `.kml` o `.kmz`. 

✏️ Para trabajar con un nuevo polígono solo requiere reemplazar la ruta de la carpeta de trabajo 


In [4]:
ruta_kml = r"C:\Users\zaira\OneDrive - Procuraduria Federal de Proteccion al Ambiente\SIIVDA\Codigos Phyton\Deforestacion - Vegetacion\Cabañas-Terrenate-Tlax_.kml"

# Verificar que el archivo existe
if not os.path.exists(ruta_kml):
    raise FileNotFoundError(f"El archivo no fue encontrado: {ruta_kml}")

# Convertir el archivo KML a GeoJSON para cargarlo en Earth Engine
gdf = gpd.read_file(ruta_kml, driver='KML')
geojson = gdf.geometry[0].__geo_interface__
aoi = ee.Geometry(geojson)
aoi_feature = ee.Feature(aoi)
aoi_fc = ee.FeatureCollection([aoi_feature])
print("✅ Polígono cargado correctamente.")


# Visualizar el polígono en un mapa interactivo con imagen ESRI de fondo
Map = geemap.Map(basemap='Esri.WorldImagery')
Map.centerObject(aoi, zoom=16)
style = {
    'color': 'red',    # Color del contorno
    'width': 3,        # Grosor de la línea
    'fillColor': '00000000'  # Relleno transparente
}
Map.addLayer(aoi_fc.style(**style), {}, 'Contorno AOI (Rojo)')
Map

✅ Polígono cargado correctamente.


Map(center=[19.481693985330033, -97.87640471671325], controls=(WidgetControl(options=['position', 'transparent…

# 4. Estimación de NDVI (Normalized Difference Vegetation Index)

El **NDVI** es un índice utilizado para el monitoreo de vegetación

## Fórmula NDVI:

$$
NDVI = \frac{(NIR - Red)}{(NIR + Red)}
$$

**Donde:**
- *NIR*: Banda infrarroja cercana (Sentinel-2: **B8**)
- *Red*: Banda roja (Sentinel-2: **B4**)

---

## Interpretación general:
- **NDVI ≈ +1** → Vegetación densa y saludable 🌿
- **NDVI ≈ 0** → Suelos desnudos o áreas sin vegetación 🏜️
- **NDVI < 0** → Cuerpos de agua, nubes u objetos no vegetales 🌊☁️


## Aplicaciones del NDVI:
a) Clasificación de vegetación.  
b) Monitoreo ambiental [(Jensen, 2000)](https://doi.org/10.1080/014311600210407).  
c) Detección de estrés en cultivos y ecosistemas [(Wang et al., 2022)](https://doi.org/10.1016/j.gecco.2022.e02034).  
d) Alerta temprana de plagas y enfermedades [(Spruce et al., 2011)](https://doi.org/10.1016/j.rse.2010.09.013).  
e) Manejo de recursos hídricos [(Aguilar et al., 2012)](https://doi.org/10.1016/j.ecolind.2012.04.008).  
f) Predicción y monitoreo de incendios forestales [(Charizanos & Demirhan, 2023)](https://doi.org/10.1016/j.ecoinf.2022.101899).




In [5]:
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)